In [2]:
import pandas as pd
import numpy as np

In [3]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')

In [6]:
import pandas as pd
import numpy as np

def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']
    orientation_x = row['orientation_x']
    orientation_y = row['orientation_y']
    orientation_z = row['orientation_z']
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check orientation for refining moving motion
            if orientation_z > 90 or orientation_z < -90:
                motion_type = "Vertical Movement"
            elif orientation_x > 90 or orientation_x < -90:
                motion_type = "Horizontal Movement"
            else:
                motion_type = "Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)


In [7]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,linear_accel_z,orientation_x,orientation_y,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type
0,2017-06-29 07:45:56.262,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
1,2017-06-29 07:46:01.263,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Moving
2,2017-06-29 07:46:01.324,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
3,2017-06-29 07:46:01.329,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,-0.033066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
4,2017-06-29 07:46:01.402,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Turning/Tilting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,2017-07-13 20:21:35.009,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
173658,2017-07-13 20:21:35.020,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
173659,2017-07-13 20:21:35.060,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary
173660,2017-07-13 20:21:35.124,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary


In [ ]:
from potassium import Potassium, Request, Response
from transformers import pipeline
import torch
import time

app = Potassium("my_app")

# @app.init runs at startup, and initializes the app's context
@app.init
def init():
    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Defind the model, using the "pipeline" interface from transformers
    bertModel = pipeline('fill-mask', model='bert-base-uncased', device=device)
   
    
    context = {
        "model_b": bertModel,
    }

    return context


def handler(context: dict, request: Request) -> Response:
    
    # Retrieve the BERT model from the context
    model = context.get("model_b")

    # Extract the motion type from the user request
    motion_type = request.json.get("motion_type")

    # Create the prompt with the motion type and [MASK] token
    prompt = f"I am {motion_type} thus [MASK]"

    # Generate text predictions using the BERT model and the prompt
    outputs = model(prompt)

    # Return the model outputs in the response
    return Response(
        json={"outputs": outputs}, 
        status=200
    )

In [8]:
from potassium import Potassium, Request, Response
from transformers import pipeline
import torch
import time

app = Potassium("my_app")

# @app.init runs at startup, and initializes the app's context
@app.init
def init():
    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Defind the model, using the "pipeline" interface from transformers
    bertModel = pipeline('fill-mask', model='bert-base-uncased', device=device)
   
    
    context = {
        "model_b": bertModel,
    }

    return context



# @app.handler is an http post handler running for every call
@app.handler("/bert")
def handler(context: dict, request: Request) -> Response:
    
    # our model is from 'model_b' as defined in the context earlier
    model = context.get("model_b")

    # our prompt is from the 'prompt' key in the user request
    prompt = request.json.get("prompt")

    # the outputs is created by passing the user request to the model
    outputs = model(prompt)

    # returned
    return Response(
        json = {"outputs": outputs}, 
        status=200
    )

if __name__ == "__main__":
    app.serve()

KeyboardInterrupt: 

In [2]:
from potassium import Potassium, Request, Response
# from transformers import pipeline
import torch

app = Potassium("my_app")

def custom_model(inputData):
  outcome = "Hello from a function, this is your input data: " + inputData
  print(outcome) 
  return outcome

# @app.init runs at startup, and initializes the app's context
@app.init
def init():

    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Define the model, using the "pipeline" interface from transformers
    # bertModel = pipeline('fill-mask', model='bert-base-uncased', device=device)

    # define any model that you understand how to process
    # model = custom_model
    
    context = {
        # "model_b": bertModel,
        # model: model
    }

    return context



# @app.handler is an http post handler running for every call
@app.handler("/custom")
def handler(context: dict, request: Request) -> Response:
    
    # our model is from 'model_b' as defined in the context earlier
    # model = context.get("model")
    # model = custom_model

    # our prompt is from the 'prompt' key in the user request
    # input = request.json.get("prompt")

    inputData = 'this is where my input data will be.'

    # the outputs is created by passing the user request to the model
    outcome = custom_model(inputData)

    print(outcome)

    # returned
    return Response(
        json = {"outputs": outcome}, 
        status=200
    )

if __name__ == "__main__":
    app.serve()

------
Starting Potassium Server 🍌
Running init()
Started 1 workers
Serving at http://0.0.0.0:8000
------
127.0.0.1 - - [22/Mar/2024 09:05:44] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [22/Mar/2024 09:05:45] "GET /favicon.ico HTTP/1.1" 405 -
127.0.0.1 - - [22/Mar/2024 09:05:58] "GET / HTTP/1.1" 405 -
